<a href="https://colab.research.google.com/github/kaveku/NLP/blob/main/Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sklearn
import numpy as np
import pandas as pd
import re
import string

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

Hello Google Drive!

In [ ]:
train  = pd.read_csv("/content/drive/MyDrive/Tweeter_sentiment_analysis_data_files/train 2.csv")
test   = pd.read_csv("/content/drive/MyDrive/Tweeter_sentiment_analysis_data_files/test 2.csv")

In [ ]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [ ]:
#Non-racist
sum(train["label"]==0)

#Racist
sum(train["label"]==1)

2242

In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import emoji

def text_preproccessing(df):  
    
    df = df.copy()
    
    def remove_URL(text):
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'',text)

    def remove_html(text):
        html=re.compile(r'<.*?>')
        return html.sub(r'',text)

    def remove_punct(text):
        table=str.maketrans('','',string.punctuation)
        return text.translate(table)

    # Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
    def remove_emoji(text):
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    df['tweet']=df['tweet'].apply(lambda x : remove_URL(x))
    df['tweet']=df['tweet'].apply(lambda x : remove_html(x))
    df['tweet']=df['tweet'].apply(lambda x : remove_punct(x))
    # cdata['text']=cdata['text'].apply(lambda x : remove_emoji(x))()


    df['tweet'] = df['tweet'].apply(lambda x : emoji.demojize(x))
    
    return df

In [ ]:
train = text_preproccessing(train)
test = text_preproccessing(test)

In [ ]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [ ]:
#Seperating the data into train and test
from sklearn.model_selection import train_test_split

# extract the labels from the train data
y = train.label.values

#Use 70% for training and 30% for testing
x_train, x_test, y_train, y_test = train_test_split(train.values, y, 
                                                    stratify=y, 
                                                    random_state=1, 
                                                    test_size=0.3, shuffle=True)

In [ ]:
#list(x_train.flatten())+list(x_test.flatten())
x_train_True=[x for x in x_train.flatten() if not type(x)==int]
x_test_True=[x for x in x_test.flatten() if not type(x)==int]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(x_train_True+x_test_True)

# transform documents to document-term matrix
x_train_vec = vectorizer.transform(x_train_True)
x_test_vec = vectorizer.transform(x_test_True)

In [ ]:
from sklearn import svm
# classify using support vector classifier
svm = svm.SVC(kernel = 'linear', probability=True)

# fit the SVC model based on the given training data
prob = svm.fit(x_train_vec, y_train).predict_proba(x_test_vec)

# perform classification and prediction on samples in x_test
y_pred_svm = svm.predict(x_test_vec)

In [ ]:
print(y_pred_svm)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy score for SVC is: ", accuracy_score(y_test, y_pred_svm) * 100, '%')

Accuracy score for SVC is:  96.36041297319846 %
